# EnvErgo project
- [researchgate link](https://www.researchgate.net/project/Isokinetic-measurements-and-musculoskeletal-modelling-for-ergonomics)
- [github link](https://github.com/romainmartinez/envergo)

_Romain Martinez_

# Model parameters

In [15]:
import matplotlib.pyplot as plt
import matplotlib.colors
import seaborn as sns
import numpy as np
import itertools
import spm1d
import scipy
import os
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42

In [16]:
path2data = '/home/romain/Desktop/envergo/parameter/'
mat = scipy.io.loadmat(os.path.join(path2data, 'Parametres.mat'))

## Data description
- `mat['PARAMETRES']`: raw model parameters (_used in the following sections_)
    - size: $35 \text{ (participants)} \times 7 \text{ (parameters)} \times 6 \text{ (positions)} \times 2 \text{ (contractions)}$
- `mat['parametres']`: model parameters converted in _SI_ units
- `mat['ERR']`: optimization residuals

<img src="../../../Downloads/envergo//ParametresModele.png" width="800">
- Expected variables in spm: $J \text{ (observations)} \times Q \text{ (frames)} \times I \text{ (variables)}$
    - `y`: $n \times 7$
    - `A`: $n \times 1$

In [17]:
nparticipant, nparameter, nposition, ncontraction = mat['PARAMETRES'].shape
y, positions = [[] for i in range(2)]
contraction = 1  # 0: flexion | 1: extension

for iparticipant in range(nparticipant):
    for iposition in range(nposition):
        y.append(mat['PARAMETRES'][iparticipant, :, iposition, contraction])
        positions.append(iposition)

y, positions = [np.array(i) for i in [y, positions]]
print('----dimension----'.upper())
funct = lambda x, y: print(f'{x} shape: {y.shape} | has {np.isnan(y).sum()} nans')
funct('y', y)
funct('positions', positions)

----DIMENSION----
y shape: (210, 7) | has 175 nans
positions shape: (210,) | has 0 nans


In [18]:
# replace missing trial by mean of the position
for iposition in np.unique(positions):
    idx = positions == iposition
    mean_vector = np.nanmean(y[idx, :], axis=0)
    y[idx,  :] = np.where(np.isnan(y[idx, :]), mean_vector, y[idx, :])
funct('y', y)
funct('positions', positions)

y shape: (210, 7) | has 0 nans
positions shape: (210,) | has 0 nans


In [19]:
def display_results(spm, kind=None, title=None, indent=''):
    print(title.upper())
    var = 'X' if kind == 'manova' else 'T'
    if spm.h0reject:
        print(f'{indent}null hypothesis rejected, {var}2={spm.z:.3f}, p={spm.p:.5f}')
    else:
        print(f'{indent}null hypothesis not rejected')

In [20]:
def spm_heatmap(df, title=None, labels=None, save_fig=False, cmap='viridis', limits=None):
    fig, axs = plt.subplots(figsize=(8, 4))
    if limits is None:
        ax = sns.heatmap(df, linewidths=.5, cbar_kws={'label': 'T statistic'},
                     annot=True, xticklabels=labels, yticklabels=labels, cmap=cmap, fmt='g')
    else:
        ax = sns.heatmap(df, linewidths=.5, cbar_kws={'label': 'T statistic'}, vmin=limits[0], vmax=limits[1],
                     annot=True, xticklabels=labels, yticklabels=labels, cmap=cmap, fmt='g')

    ax.invert_yaxis()
    ax.set_title(title)
    plt.show()
    
    if save_fig:
        fig.savefig('./share/fig/{}.pdf'.format(title), transparent=True)

## One-way MANOVA (non-parametric)

In [21]:
alpha = 0.05
iterations = 10000
np.random.seed(0)

m_spm = spm1d.stats.nonparam.manova1(y, positions)
m_spmi = m_spm.inference(alpha=alpha, iterations=iterations)

display_results(m_spmi, title='one-way manova', kind='manova')

ONE-WAY MANOVA
null hypothesis rejected, X2=99.206, p=0.00010


- There was a significant main effect for position ($p<0.001$)
- As the manova reach significance, we can conduct separate Hotelling’s T$^2$ tests on each pair of groups
    - If those tests reach significance then conduct additional post hoc t tests on each vector component, but acknowledge that this neglects vector component covariance.

## Post hoc analysis (non-parametric paired Hotellings's T2 and paired t-test)

In [ ]:
n_positions = len(set(positions))
combinations = itertools.combinations(range(n_positions), 2)
names_positions = ['Ref', 'Abd90IR', 'Flex90IR', 'Flex180ER', 'Abd90ER', 'Flex90ER']
names_parameters = ['Max. isometric torque', 'Max. isometric range of motion', 'Isometric optimal range', 'Velocity at max. power',
                   'Max. conc. velocity', 'Max. ecc. velocity', 'Max. ecc. to conc. torque ratio']
H_heatmap = np.full([6, 6], np.nan)
T_heatmap = np.full([6, 6, 7], np.nan)

plot_diff = True

for A, B in combinations:
    yA = y[positions == A]
    yB = y[positions == B]
    
    h_spm = spm1d.stats.nonparam.hotellings_paired(yA, yB)
    h_spmi = h_spm.inference(alpha, iterations=iterations, force_iterations=True)
    
    display_results(h_spmi, title=f'hotelling {names_positions[A]} vs. {names_positions[B]}', kind='hotelling')
    
    if h_spmi.h0reject:
        H_heatmap[B, A] = int(h_spmi.z)
        for iparameter in range(nparameter):
            t_spm = spm1d.stats.nonparam.ttest_paired(yA[:, iparameter], yB[:, iparameter])
            t_spmi = t_spm.inference(alpha, two_tailed=True, iterations=iterations, force_iterations=True)

            display_results(t_spmi, title=f'\tt-test parameter {names_parameters[iparameter]}', kind='t-test', indent='\t')
            
            if t_spmi.h0reject:
                if plot_diff:
                    # mean difference
                    #T_heatmap[B, A, iparameter] = (np.mean(yA[:, iparameter]) - np.mean(yB[:, iparameter]))
                    # mean percentage difference
                    T_heatmap[B, A, iparameter] = (100 * (np.mean(yA[:, iparameter]) - np.mean(yB[:, iparameter])) / np.mean(yB[:, iparameter])).astype(int)
                else:
                    T_heatmap[B, A, iparameter] = t_spmi.z
    print('--------')

HOTELLING REF VS. ABD90IR
null hypothesis rejected, T2=37.554, p=0.00110
	T-TEST PARAMETER MAX. ISOMETRIC TORQUE
	null hypothesis rejected, T2=2.191, p=0.01280
	T-TEST PARAMETER MAX. ISOMETRIC RANGE OF MOTION
	null hypothesis not rejected
	T-TEST PARAMETER ISOMETRIC OPTIMAL RANGE
	null hypothesis rejected, T2=-2.962, p=0.00700
	T-TEST PARAMETER VELOCITY AT MAX. POWER
	null hypothesis rejected, T2=2.740, p=0.00660
	T-TEST PARAMETER MAX. CONC. VELOCITY
	null hypothesis rejected, T2=2.798, p=0.00540
	T-TEST PARAMETER MAX. ECC. VELOCITY
	null hypothesis not rejected
	T-TEST PARAMETER MAX. ECC. TO CONC. TORQUE RATIO
	null hypothesis rejected, T2=3.036, p=0.00140
--------
HOTELLING REF VS. FLEX90IR
null hypothesis rejected, T2=25.328, p=0.01080
	T-TEST PARAMETER MAX. ISOMETRIC TORQUE
	null hypothesis rejected, T2=2.478, p=0.01880
	T-TEST PARAMETER MAX. ISOMETRIC RANGE OF MOTION
	null hypothesis not rejected
	T-TEST PARAMETER ISOMETRIC OPTIMAL RANGE
	null hypothesis not rejected
	T-TEST PARAM

In [ ]:
spm_heatmap(H_heatmap, title='Hotelling T statistics for each position against each other',
            labels=names_positions, save_fig=True, cmap='viridis', limits=[0, 100])

In [ ]:
limits = [-100, 100]
colors = ['#7570b3', 'white', '#1b9e77']
cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", colors)
for iparameter in range(nparameter):
    spm_heatmap(T_heatmap[:, :, iparameter], title=f'T statistics for each position against each other [{names_parameters[iparameter]}]',
                labels=names_positions, save_fig=True, limits=limits, cmap=cmap)